# Laboratorio 7: Clasificación 🤗

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>


### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliares: Catherine Benavides y Consuelo Rojas
- Ayudante: Nicolás Ojeda, Eduardo Moya

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Vanessa González
- Nombre de alumno 2: Benjamín Angulo

### Temas a tratar
- Clasificación en problemas desbalanceados
- Lightgbm y xgboost
- Pipelines

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.



### Objetivos principales del laboratorio
- Comprender cómo trabajar con problemas de clasificación con clases desbalanceadas.
- Aplicar los modelos lightgbm y xgboost.
- Practicar Pipelines

# Parte Teórica [12 puntos]




1. Explique cuál es la diferencia entre los datos de entrenamiento y validación. [1 punto]

2. Explique cuál es el principal desafío al trabajar problemas de clasificación con data no supervisada. [1 punto]

3. Explique en **sus palabras** qué es la matriz de confusión y para qué se utiliza. [1 puntos]

4. Escriba la fórmula de las siguientes métricas y explique con **sus palabras** cómo se interpretan. [1 punto cada uno]

  * Accuracy
  * Precision
  * Recall
  * F1 score

5. Explique qué métrica recomendaría para los siguientes contextos de clasificación. [1 punto cada uno]

  * Mantenimiento predictivo de fallas de maquinaria pesada en la industria minera.  
  * Detección de enfermedades altamente contagiosas.
  * Aprobación de créditos de alto riesgo.
  * Detección de crímenes.

6. Explique qué es la calibración de modelos y para qué se usa. [1 punto]


**Respuesta:**

*Escriba su respuesta aquí*

# Parte práctica [48 puntos]

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones.

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con el csv `statsplayers.csv`, donde deberán aplicar algoritmos de aprendizaje supervisado de clasificación en base a características que describen de jugadores de fútbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except:
    print('Ignorando conexión drive-colab')

In [164]:
import pandas as pd

df = pd.read_csv("stats_players.csv")

## 1. Predicción de Seleccionados Nacionales [14 puntos]

<p align="center">
  <img src="https://www.futuro.cl/wp-content/uploads/2016/06/chile-argentina-meme-12.jpg" width="300">
</p>



### 1.1 Preprocesamiento [5 puntos]

Tareas:

1. Genere los labels para la clasificación binaria en una variable llamada `label`. Para esto, trabaje sobre el atributo `National_Position` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. [Sin puntaje]

2. Hecho esto, ¿cuántos se tienen ejemplos por cada clase? Comente lo que observa. [1 punto]

3. Genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guárdelo una variable llamada `col_transformer`. [2 puntos]

4. Comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material `Player_Stats_Report.html` que viene en el zip del lab), al igual que las transformaciones aplicadas. [2 puntos]

**Respuesta:**

In [165]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler

df["label"] = df["National_Position"].notnull().astype(int) # Se crea el atributo binario "label" 

class_count = df["label"].value_counts() # Se hace un conteo de la cantidad de datos por clase del atributo "label"
print(class_count)

categorical_data = ["Club_Position", "Preffered_Foot", "Work_Rate"] # Se crea una lista con las variables categoricas, eliminando las que no aportan información como el nombre, nacionalidad y national position.
numerical_data_standard = ["Height", "Weight", "Age", "Ball_Control", "Dribbling", "Marking", "Sliding_Tackle", "Standing_Tackle", "Aggression", "Reactions", "Interceptions", "Vision", "Composure", "Crossing", "Short_Pass", "Long_Pass", "Acceleration", "Speed", "Stamina", "Strength", "Balance", "Agility", "Jumping", "Heading", "Shot_Power", "Finishing", "Long_Shots", "Curve", "Freekick_Accuracy", "Penalties", "Volleys"] # Se crea una lista con las variables numéricas con las que se usará la transformación StandardScaler
numerical_data_minmax = ["Weak_foot", "Skill_Moves"] # Se crea una lista con las variables numéricas con las que se usará la transformación MinMaxScaler

col_transformer = ColumnTransformer(
    transformers=[("categorical", OneHotEncoder(), categorical_data),
                  ("numerical_standard", StandardScaler(), numerical_data_standard),
                  ("numerical_minmax", MinMaxScaler(), numerical_data_minmax)]) # Se crea el ColumnTransformer con las transformaciones esperadas para cada conjunto de variables

label
0    16513
1     1075
Name: count, dtype: int64


### 1.2 Entrenamiento [3 puntos]

Ahora, vamos a entrenar los pipelines generados en los pasos anteriores. Para esto, debe realizar las siguientes tareas:

1. Separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio). En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`). [1 puntos]


2. Defina un pipeline llamado `pipeline_xgboost` y otro llamado `pipeline_lightgbm`. Estos pipelines deben tener el mismo ColumnTransformer definido en la sección de preprocesamiento, pero deben variar los clasificadores de acuerdo al nombre de cada pipeline. [1 puntos]

3. Entrene los pipelines. [1 punto]



**Respuesta:**

In [144]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Se separan las variables de las clases 
X = df.drop("label", axis=1) 
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y) # Se crean los sets de entrenamiento y prueba considerando un 20% de los datos para prueba y manteniendo la proporción de clases


pipeline_xgboost = Pipeline([("transform", col_transformer), 
                             ("classifier", XGBClassifier())]) # Se crea la pipeline para el XGBClassifier

pipeline_lightgbm = Pipeline([("transform", col_transformer),
                              ("classifier", LGBMClassifier())]) # Se crea la pipeline para le LGBMClassifier

# Se entrenan los pipelines
pipeline_xgboost.fit(X_train, y_train) 
pipeline_lightgbm.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 860, number of negative: 13210
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001448 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2526
[LightGBM] [Info] Number of data points in the train set: 14070, number of used features: 70
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.061123 -> initscore=-2.731797
[LightGBM] [Info] Start training from score -2.731797


Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('categorical',
                                                  OneHotEncoder(),
                                                  ['Club_Position',
                                                   'Preffered_Foot',
                                                   'Work_Rate']),
                                                 ('numerical_standard',
                                                  StandardScaler(),
                                                  ['Height', 'Weight', 'Age',
                                                   'Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Aggression', 'Reactions',
                                                   'Interceptions', 'Vision',
                                                   'Composure', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Acceleration', 'Speed',
                                                   'Stamina', 'Strength',
                                                   'Balance', 'Agility',
                                                   'Jumping', 'Heading',
                                                   'Shot_Power', 'Finishing',
                                                   'Long_Shots', 'Curve',
                                                   'Freekick_Accuracy',
                                                   'Penalties', ...]),
                                                 ('numerical_minmax',
                                                  MinMaxScaler(),
                                                  ['Weak_foot',
                                                   'Skill_Moves'])])),
                ('classifier', LGBMClassifier())])

### 1.3 Resultados [6 puntos]

1. Calcule las métricas accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional) para evaluar el rendimiento de los distintos modelos. Verifique sus resultados usando `classification_report`. [2 puntos]

2. Explique qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional) y cómo influye la cantidad de ejemplos por clase en los resultados obtenidos. [2 puntos]

3. Explique qué métrica le parece más adecuada y concluya qué modelo tiene un mejor desempeño. [2 puntos]

**Respuesta:**

In [145]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

# Se realizan las predicciones para ambos modelos con el set de prueba
pred_xgboost = pipeline_xgboost.predict(X_test)
pred_lightgbm = pipeline_lightgbm.predict(X_test)

# Se calculan el accuracy, precision y recall del modelo xgboost
accuracy_xgboost = accuracy_score(y_test, pred_xgboost)
precision_xgboost = precision_score(y_test, pred_xgboost)
recall_xgboost = recall_score(y_test, pred_xgboost)

# Se calculan el accuracy, precision y recall del modelo lightgbm
accuracy_lightgbm = accuracy_score(y_test, pred_lightgbm)
precision_lightgbm = precision_score(y_test, pred_lightgbm)
recall_lightgbm = recall_score(y_test, pred_lightgbm)

# Se realizan los classification report para ambos modelos
report_xgboost = classification_report(y_test, pred_xgboost)
report_lightgbm = classification_report(y_test, pred_lightgbm)

print("Resultados para modelo xgboost:")
print(f'Accuracy: {accuracy_xgboost}')
print(f'Precision: {precision_xgboost}')
print(f'Recall: {recall_xgboost} \n')

print("Resultados para modelo lightgbm:")
print(f'Accuracy: {accuracy_lightgbm}')
print(f'Precision: {precision_lightgbm}')
print(f'Recall: {recall_lightgbm} \n')

print("Classification report para ambos modelos:")
print(f'Xgboost: \n {report_xgboost}')
print(f'Lightgbm: \n {report_lightgbm}')

Resultados para modelo xgboost:
Accuracy: 0.939169982944855
Precision: 0.5074626865671642
Recall: 0.15813953488372093 

Resultados para modelo lightgbm:
Accuracy: 0.9400227401932917
Precision: 0.5370370370370371
Recall: 0.13488372093023257 

Classification report para ambos modelos:
Xgboost: 
               precision    recall  f1-score   support

           0       0.95      0.99      0.97      3303
           1       0.51      0.16      0.24       215

    accuracy                           0.94      3518
   macro avg       0.73      0.57      0.60      3518
weighted avg       0.92      0.94      0.92      3518

Lightgbm: 
               precision    recall  f1-score   support

           0       0.95      0.99      0.97      3303
           1       0.54      0.13      0.22       215

    accuracy                           0.94      3518
   macro avg       0.74      0.56      0.59      3518
weighted avg       0.92      0.94      0.92      3518



## 2. Predicción de posiciones de jugadores [4 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="300">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas.

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características.

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo `Club_Position`, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF']
central_ataque = ['RW', 'CAM', 'LW']
central = ['RM', 'CM', 'LM']
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

La elección del clasificador se justificar en base a la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y se deben comentar los resultados obtenidos en la clasificación.

**Tareas:** [1 punto por tarea]

1. Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
2. Cuente cuántos por clase quedan.
3. Entrene el nuevo pipeline y ejecute una evaluación de este.  
4. Comente los resultados obtenidos.

**Respuesta:**

In [166]:
mapa = {
    "ST": "ataque", "CF": "ataque",
    "RW": "central_ataque", "CAM": "central_ataque", "LW": "central_ataque", 
    "RM": "central", "CM": "central", "LM": "central", 
    "RWB": "central_defensa", "CDM": "central_defensa", "LWB": "central_defensa", 
    "RB": "defensa", "CB": "defensa", "LB": "defensa", 
    "GK": "arquero"
}

df["Club_Position"] = df["Club_Position"].map(mapa)

conteo = df['Club_Position'].value_counts()
print(conteo)

Club_Position
defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: count, dtype: int64


In [172]:
print(df.columns)

Index(['Name', 'Nationality', 'National_Position', 'Club_Position', 'Height',
       'Weight', 'Preffered_Foot', 'Age', 'Work_Rate', 'Weak_foot',
       'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys', 'label'],
      dtype='object')


In [192]:
from sklearn.svm import LinearSVC

df = df.dropna(subset=["Club_Position"])
X = df.drop(["Club_Position"], axis=1)
y = df["Club_Position"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

categorical_data = ["Preffered_Foot", "Work_Rate"] # Se crea una lista con las variables categoricas, eliminando las que no aportan información como el nombre, nacionalidad y national position. Además se elimina el objetivo a predecir "Club_Position"
numerical_data_standard = ["Height", "Weight", "Age", "Ball_Control", "Dribbling", "Marking", "Sliding_Tackle", "Standing_Tackle", "Aggression", "Reactions", "Interceptions", "Vision", "Composure", "Crossing", "Short_Pass", "Long_Pass", "Acceleration", "Speed", "Stamina", "Strength", "Balance", "Agility", "Jumping", "Heading", "Shot_Power", "Finishing", "Long_Shots", "Curve", "Freekick_Accuracy", "Penalties", "Volleys"] # Se crea una lista con las variables numéricas con las que se usará la transformación StandardScaler
numerical_data_minmax = ["Weak_foot", "Skill_Moves"] # Se crea una lista con las variables numéricas con las que se usará la transformación MinMaxScaler

col_transformer = ColumnTransformer(
    transformers=[("categorical", OneHotEncoder(), categorical_data),
                  ("numerical_standard", StandardScaler(), numerical_data_standard),
                  ("numerical_minmax", MinMaxScaler(), numerical_data_minmax)]) # Se crea el ColumnTransformer con las transformaciones esperadas para cada conjunto de variables

pipeline_linearsvc = Pipeline([("transform", col_transformer),
                              ("classifier", LinearSVC())])

pipeline_linearsvc.fit(X_train, y_train)

pred_linearsvc = pipeline_linearsvc.predict(X_test)

report_linearsvc = classification_report(y_test, pred_linearsvc)

print("Classification report para modelo LinearSVC:")
print(report_linearsvc)

c:\Users\benja\OneDrive\Escritorio\MDS7202\venv\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Classification report para modelo LinearSVC:
                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       126
         ataque       0.81      0.92      0.86        86
        central       0.60      0.65      0.62       182
 central_ataque       0.62      0.33      0.43       116
central_defensa       0.33      0.10      0.15        42
        defensa       0.78      0.97      0.86       236

       accuracy                           0.75       788
      macro avg       0.69      0.66      0.65       788
   weighted avg       0.73      0.75      0.73       788



c:\Users\benja\OneDrive\Escritorio\MDS7202\venv\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


## 3. Predicciones de Seleccionados Nacionales para el Jere Klein [30 puntos]

<center>
<img src='https://www.radioactiva.cl/wp-content/uploads/2024/04/Jere-Klein-1-768x432.webp' width=500 />

Después de alcanzar la fama como cantante urbano, Jere Klein decide explorar una nueva faceta. Con su amor por el fútbol y convencido de que los artistas urbanos poseen un talento y versatilidad excepcionales, Jere se embarca en un proyecto innovador: desarrollar un sistema de inteligencia artificial capaz de identificar a jugadores que tienen potencial para convertirse en futbolistas profesionales. Su teoría es que muchos artistas del género urbano chileno, con sus habilidades únicas y su disciplina, podrían destacarse también en el deporte. Con este sistema, Jere espera no solo abrir nuevas oportunidades para sus colegas artistas, sino también demostrar la amplia gama de talentos que pueden ofrecer.

### 2.1 ¿Qué modelo de árbol es más de "pana"? [10 puntos]

<center>
<img src='https://64.media.tumblr.com/39189215a7d3d96823cb359f35b44e05/tumblr_psmrhrR3Xw1qf5hjqo4_540.gif' width=300 />


**Tareas**


1. Considerando el la variable llamada `label` creada en la sección 1.1. Para determinar cuál modelo de árbol sería más adecuado para la tarea en cuestión, utilice PyCaret. Este deberá centrarse exclusivamente en modelos de tipo árbol. Jere ha especificado que busca un modelo que tome decisiones rápidamente y que tenga una baja tasa de falsos positivos, ya que planea invertir en estos jugadores. [3 puntos] Para la comparación, utilice los siguientes modelos:

```python
['et', 'rf', 'dt', 'xgboost', 'lightgbm', 'catboost']
```

3. Explique en brevemente que son los modelos de la siguiente lista `['et', 'rf', 'dt']` y como funcionan. [3 punto]

4. Tras realizar la comparación de modelos, seleccione aquel que muestre el mejor rendimiento en términos de velocidad y precisión, especialmente en la reducción de falsos positivos. Utilice la función `evaluate_model` de PyCaret para revisar y analizar los resultados obtenidos en los siguientes aspectos:

  - **Confusión Matrix**: ¿Cómo se encuentran la tasa de verdaderos positivos y verdaderos negativos?
  - **Threshold**: ¿Es acaso el umbral por defecto del modelo el mejor para las predicciones?
  - **Feature Importance**: ¿Cuáles son las variables con mejor desempeño? ¿A qué podría deberse esto?
  - **Learning Curve**: ¿El modelo presenta algún problema?

  [4 puntos]

**Respuesta**

### 2.2 Reducción de dimensionalidad [14 puntos]

<center>
<img src='https://i.kym-cdn.com/photos/images/original/002/258/560/668.gif' width=400 />

A pesar de los resultados obtenidos previamente, el manager de Jere ha solicitado el entrenamiento de un modelo de XGBoost utilizando los datos disponibles. Además, se debe proceder a realizar una reducción de dimensionalidad basada en la importancia de las características.

Para llevar a cabo esta tarea:

1. Inicie entrenando un modelo XGBoost con todas las características disponibles. [2 puntos]

2. Una vez el modelo esté entrenado, evalúe y clasifique las características según su importancia de forma descendente. [2 puntos]

3. Utilice esta clasificación para ejecutar una búsqueda recursiva de eliminación de características, eliminando progresivamente las menos importantes y evaluando el impacto en el desempeño del modelo hasta identificar las N características más críticas. [2 puntos]

4. Con este conjunto reducido de características, entrene un nuevo modelo y evalúe su rendimiento. [2 puntos]

5. Posteriormente, responda a las siguientes preguntas para una comprensión más profunda de los cambios y beneficios:

  - ¿El rendimiento del modelo con las características seleccionadas es similar al del modelo original? ¿Cómo se comparan en términos de precisión y robustez? [2 puntos]
  - ¿Cuáles son los beneficios potenciales de eliminar variables del modelo? Considere factores como la simplificación del modelo, reducción del tiempo de entrenamiento, y mejora en la capacidad de generalización. [2 puntos]
  - Comente si el modelo con menor dimensionalidad es más sencillo de explicar. Explique brevemente por qué la eliminación de ciertas características puede facilitar la comprensión y la explicación del comportamiento del modelo. [2 puntos]

Notar que con esta metodologia buscamos encontrar un punto entermedio entre número de festures y desempeño. por esto, si observa que al aumentar festires el aumento es despreciable, puede no considerar agregar más features a su modelo.

**Respuesta**

### 2.3 Calibración Probabilistica [6 puntos]

<center>
<img src='https://media2.giphy.com/media/l2Je4Ku0Cx292KWv6/200w.gif?cid=6c09b952y0sihtq9tb6sz8j2023x3zxxp3qx1ocgonkpkblj&ep=v1_gifs_search&rid=200w.gif&ct=g' width=400 />

Para lograr modelos más modulares, se recomienda realizar una calibración del modelo entrenado anteriormente, con el objetivo de obtener salidas que reflejen mayor modularidad.

1. Se solicita que utilice un método de calibración que asegure que las probabilidades generadas incrementen de manera monótona. Una métrica ampliamente utilizada para evaluar la precisión de la calibración de un modelo es el Brier Score. Calcule el Brier Score para el modelo tanto antes como después de la calibración. Esto le permitirá realizar una comparación cuantitativa y determinar si la calibración ha mejorado el rendimiento del modelo. Para más información sobre el Brier Score, puede consultar el siguiente enlace: [Scikit-Learn - Brier Score Loss](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.brier_score_loss.html). [3 puntos]

2. Tras la calibración, examine y comente los resultados obtenidos. A su análisis añada una comparación visual de las ideales versus las salidas del modelo original (sin calibrar) y del modelo calibrado. [3 puntos]

**Respuesta**